In [12]:
from IPython.display import IFrame

url = 'https://www.99acres.com/property-in-gurgaon-ffid-page'
IFrame(url, height=500, width=1080)

In [2]:
# Importing Libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [3]:
# Headers set like below:
# User Agent
headers = {
    'authority': 'www.99acres.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': 'https://www.99acres.com/property-in-gurgaon-ffid',
    'sec-ch-ua': '"Chromium";v="107", "Not;A=Brand";v="8"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/527.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}

In [4]:
# Extract function

def extract_data(pageSoup):
    global i
    d = pd.DataFrame()
    for soup in pageSoup.select_one('div[data-label="SEARCH"]').select('section[data-hydration-on-demand="true"]'):

        # Extract property name and property sub-name
        try:
            property_name = soup.find('a', class_='projectTuple__projectName').text.strip()
            property_sub_name = soup.find('h2', class_='projectTuple__subHeadingWrap').text.strip()
            # print(property_name+'\n'+property_sub_name)
            # Extract link
            link = soup.select_one('a', class_='projectTuple__projectName')['href']

            page = requests.get(link, headers=headers)
            dpageSoup = BeautifulSoup(page.content, 'html.parser')
            top_f=[]
            top_facilities = dpageSoup.find('div',id='top-facilities').find_all('div', class_="UniquesFacilities__xidFacilitiesCard")
            for facilities in top_facilities :
                top_f.append(facilities.text.strip())

            # print(top_f)
            # Extract Nearbay Locations with Distances
            LocationAdvantages = {}
            for l in dpageSoup.select_one('div[data-label="LOCATION_HIGHLIGHTS"]').select('div.locAdvantagesCard__locAdCard'):
                t = l.find('div').find_all('div')
                loaction = t[0].text.strip()
                distance = t[1].text.strip()
                LocationAdvantages[loaction] = distance

            # Extract nearby locations
            nearby_elements = soup.find_all('div', class_="SliderTagsAndChips__container")[0].find_all('li', class_ = 'SliderTagsAndChips__item')
            nearby = [element.text.strip() for element in nearby_elements]


            #price Range
            price_range = soup.find('div', class_="pageComponent configurationCards__srpCardStyle").text

            # Extract price details
            prices_details = {}
            price_elements =  soup.find('div', class_ = 'carousel__CarouselBox').find_all('div', class_="configurationCards__cardContainer")
            for element in price_elements:
                bedroom_type = element.select_one('span.configurationCards__configBandLabel').text.strip()
                building_type = element.select_one('span.configurationCards__configBandHeading').text.strip()
                area_type = element.select_one('span.configurationCards__cardAreaTypeStyle').text.strip()
                area = element.select_one('span.configurationCards__cardAreaSubHeadingOne').text.strip()
                price_range = element.select_one('span.configurationCards__cardPriceHeading').text.strip()

                prices_details[bedroom_type] = {
                    'building_type': building_type,
                    'area_type' : area_type,
                    'area': area,
                    'price-range': price_range
                }
            # # Print the extracted data
            # print("Property Name: ", property_name)
            # print("Property Sub-name: ", property_sub_name)
            # print("Nearby Locations: ", nearby)
            # print("Location Advantages: ", LocationAdvantages)
            # print("Link: ", link)
            # print("Price Details: ", prices_details)
            # print('Top Facilities: ', top_f)


            # Create a dictionary with the given variables
            data_dict = {
                "PropertyName": property_name,
                "PropertySubName": property_sub_name,
                "NearbyLocations": nearby,
                "LocationAdvantages": LocationAdvantages,
                "Link": link,
                "PriceDetails": prices_details,
                "TopFacilities": top_f
            }

            temp_df = pd.DataFrame.from_records([data_dict])
            # print(temp_df)
            d = pd.concat([d, temp_df], ignore_index=True)


        except:
            # print('No Data')
            pass
        i += 1
    return d

In [5]:
pageNumber=1
i=1
# Create an empty DataFrame
df = pd.DataFrame()

# Specify the file path for the CSV file
file_path = "appartment_data.csv"
# df.to_csv(file_path, mode='a', index=False)

while pageNumber < 50:
    URL = f'https://www.99acres.com/property-in-gurgaon-ffid-page-{pageNumber}'
    page = requests.get(URL, headers=headers)
    pageSoup = BeautifulSoup(page.content, 'html.parser')
    try:
        data = extract_data(pageSoup)

        # Append the dictionary as a row in the DataFrame
        if df.empty:
            df = pd.concat([data, df], ignore_index=True)
            data.to_csv(file_path, index=False)
        else:
            df = pd.concat([data, df], ignore_index=True)
            data.to_csv(file_path, mode='a', index=False, header=False)

        print(f"Data Extracted from {pageNumber}  : {data.shape}. Total Data : {df.shape} ")
        pageNumber += 1
    except:
        print("Request Might be decline- waiting for 50 sec to request again.")
        time.sleep(50)
# print(soup.prettify())

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [7]:
df.shape

(0, 0)

In [9]:
data = pd.read_csv('/WebScrap Data/appartment_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/WebScrap Data/appartment_data.csv'

In [ ]:
data.shape

(268, 7)

In [ ]:
data.drop_duplicates()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Whiteland The Aspen,"3, 4, 5 BHK Apartment in Sector 76, Gurgaon","['Vega Schools NH-8', 'DLF Corporate Greens', ...","{'Vega Schools NH-8': '650 Meter', 'DLF Corpor...",https://www.99acres.com/whiteland-the-aspen-se...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Reflexology Park', 'Card Room', 'High Speed ..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,M3M Antalya Hills,"2, 3 BHK Apartment in Sector 79 Gurgaon","['Nakhrola Stadium', 'Delhi - Jaipur Expresswa...","{'Nakhrola Stadium': '4.9 Km', 'Delhi - Jaipur...",https://www.99acres.com/m3m-antalya-hills-sect...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Mini Theatre', 'Swimming Pool', 'Lounge', 'R..."
3,Elan The Presidential,"3, 4, 5 BHK Apartment in Sector 106, Gurgaon","['Dwarka Expressway', 'Euro International Scho...","{'Dwarka Expressway': '1.6 KM', 'Euro Internat...",https://www.99acres.com/elan-the-presidential-...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Aerobics Centre', 'Shopping..."
4,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Palam Vihar Halt', 'DPSG Palam Vihar', 'Park...","{'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vih...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
...,...,...,...,...,...,...,...
263,Alpha Corp GurgaonOne 84,"2, 3, 4 BHK Apartment in Sector 84, Gurgaon","['Cambridge Pre-School', 'Euro Int. School', '...","{'Cambridge Pre-School': '550 m', 'Euro Int. S...",https://www.99acres.com/alpha-corp-gurgaonone-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Medical Centre', 'Flower Ga..."
264,Krrish Florence Estate,"3 BHK Apartment in Sector 70, Gurgaon","['The Vivekananda School', 'Suraj PG Degree Co...","{'The Vivekananda School': '600 Meter', 'Suraj...",https://www.99acres.com/krrish-florence-estate...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Infinity Pool', 'Swimming Pool', 'Laundry', ..."
265,Unitech Fresco,"2, 3 BHK Apartment in Nirvana Country, Gurgaon","[""McDonald's India"", 'Park Hospital', 'Golf Co...","{""McDonald's India"": '1.8Km', 'Park Hospital':...",https://www.99acres.com/unitech-fresco-nirvana...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'School', 'ATM', 'Piped Gas'..."
266,BPTP Freedom Park Life,"2, 3, 5 BHK Apartment in Sector 57, Gurgaon","['Presidium School Gurgoan', 'Manav Rachna Sch...","{'Presidium School Gurgoan': '1min away', 'Man...",https://www.99acres.com/bptp-freedom-park-life...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'School', 'Jacuzzi', 'Shoppi..."
